In [1]:
import spectraltree
import time
import numpy as np

## Generate data according to a predefined tree model

In [2]:


num_taxa = 128   # Number of terminal nodes
n = 1200        # Number of independent samples (sequence length)   
jc = spectraltree.Jukes_Cantor()   #set evolution process to the Jukes Cantor model
mutation_rate = jc.p2t(0.9)        #set mutation rate between adjacent nodes to 1-0.9=0.1


# create a tree according to the coalescent model
reference_tree = spectraltree.unrooted_pure_kingman_tree(num_taxa)

# create a tree according to the birth death model model        
#reference_tree = spectraltree.unrooted_birth_death_tree(num_taxa)

# create a symmetric binary tree
#reference_tree = spectraltree.balanced_binary(num_taxa)

# create a caterpiller tree 
#reference_tree = spectraltree.lopsided_tree(num_taxa)

# generate sequences: input - sequence length, specified tree, evolutionary model, mutation rate and alphabet
observations, taxa_meta = spectraltree.simulate_sequences(n, tree_model=reference_tree, seq_model=jc, mutation_rate=mutation_rate, alphabet="DNA")



## Recover tree by NJ and STDR + NJ


In [3]:

# run NJ to recover the full tree
nj = spectraltree.NeighborJoining(spectraltree.JC_similarity_matrix)
time_start = time.time()
tree_nj = nj(observations, taxa_meta)
nj_runtime = time.time()-time_start

In [4]:
stdr_nj = spectraltree.STDR(spectraltree.NeighborJoining,spectraltree.JC_similarity_matrix)   

time_start = time.time()
tree_stdr_nj = stdr_nj(observations, 
        taxa_metadata= taxa_meta, 
        threshold = 64,
        min_split = 1,
        merge_method = "least_square", 
        verbose=False)
stdr_nj_runtime = time.time()-time_start

In [5]:
# print results and runtime
# compare output of NJ to reference tree
RF_nj,F1 = spectraltree.compare_trees(tree_nj, reference_tree)
print('Normalized RF for NJ:',np.round_(RF_nj/(2*num_taxa-6),2),' runtime: ', np.round_(nj_runtime,2),'s')

# compare output of STDR + NJ to reference tree
RF_stdr_nj,F1 = spectraltree.compare_trees(tree_stdr_nj, reference_tree)
print('Normalized RF for STDR+NJ: ',np.round_(RF_stdr_nj/(2*num_taxa-6),2), '  runtime: ',np.round_(stdr_nj_runtime,2),'s')

Normalized RF for NJ: 0.28  runtime:  0.42 s
Normalized RF for STDR+NJ:  0.22   runtime:  0.15 s


## Recover tree by RAxML and STDR + RAxML


In [6]:

# run RAxML to recover the full tree
raxml = spectraltree.RAxML()
time_start = time.time()
tree_raxml = raxml(observations, taxa_meta)
raxml_runtime = time.time()-time_start


In [7]:
# run STDR with RAxML as subroutine
stdr_raxml = spectraltree.STDR(spectraltree.RAxML,spectraltree.JC_similarity_matrix)   

time_start = time.time()
tree_stdr_raxml = stdr_raxml(observations, 
        taxa_metadata= taxa_meta, 
        threshold = 32,
        min_split = 5,
        merge_method = "least_square", 
        verbose=False)
stdr_raxml_runtime = time.time()-time_start

In [8]:
# print results and runtime
# compare output of SNJ to reference tree
RF_raxml,F1 = spectraltree.compare_trees(tree_raxml, reference_tree)
print('Normalized RF for RAxML:',np.round_(RF_raxml/(2*num_taxa-6),2),' runtime: ', np.round_(raxml_runtime,2),'s')

# compare output of NJ to reference tree
RF_stdr_raxml,F1 = spectraltree.compare_trees(tree_stdr_raxml, reference_tree)
print('Normalized RF for STDR+RAxML: ',np.round_(RF_stdr_raxml/(2*num_taxa-6),2), '  runtime: ',np.round_(stdr_raxml_runtime,2),'s')

Normalized RF for RAxML: 0.24  runtime:  31.47 s
Normalized RF for STDR+RAxML:  0.22   runtime:  17.07 s
